# Wikipédia madárcsaládok 

<b>Feladat:</b>
Gyűjtsünk alapvető adatokat az egyes madárcsaládokról. A Wikipédián elérhető gyűjtőoldalon linkek találhatók a madárcsaládok egyes képviselőire. A linkek begyűjtése után használjuk fel azokat a konkrét weboldalakról származó (pl.: cím, leírás, kép, stb.) adatok kinyerésére. Ha ez megvan, készítsünk egy összefoglaló weboldalt az adatok segítségével. 

Mielőtt nekikezdenénk a program fejlesztésének, ismerkedjünk meg a weboldallal:

<b>Madárcsaládok:</b> https://hu.wikipedia.org/wiki/Kateg%C3%B3ria:Mad%C3%A1rcsal%C3%A1dok

### Szükséges könyvtárak telepítése és importálása

In [1]:
#%pip install bs4
#%pip install requests

In [2]:
from bs4 import BeautifulSoup
import requests as re
import time
import os

## Kapcsolat létesítése a választott weboldallal

200-as kód sikert jelöl

In [3]:
url = "https://hu.wikipedia.org/wiki/Kateg%C3%B3ria:Mad%C3%A1rcsal%C3%A1dok"

response = re.get(url)
print(response.status_code)

if response.status_code == 200:
    print("Siker!")

200
Siker!


## <code>print()</code> limitálása

Az adatbányászat témakörnél előfordul, hogy egy egyszerű <code>print()</code> utasítás kimenete terjedelmes és rontja a jegyzet olvashatóságát. Az alábbi függvény segítségével limitáljuk a kimenetet. Általában 1000 karakterből el tudjuk dönteni a kimenet helyességét, de szükség esetén könnyen növelhető, csökkenthető.

<code>obj</code> - mit szeretnénk kiíratni?

<code>length</code> - hány karakternyit szeretnék kiíratni?

In [4]:
def printLimited(obj, length):
    print(str(obj)[:length])

## A letöltött HTML struktúra feldolgozása

### A teljes HTML struktúra megtekintése

A weboldal struktúráját a <code>soup</code> nevű változó fogja tárolni.

In [5]:
soup = BeautifulSoup(response.text, "html.parser")
printLimited(soup,1000)

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-excluded vector-toc-not-available" dir="ltr" lang="hu">
<head>
<meta charset="utf-8"/>
<title>Kategória:Madárcsaládok – Wikipédia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-

### Szűrés pontosítása konkrét adatokra

Ilyen jellegű munkánál érdemes a teljes weboldal struktúráját tartalmazó változót értintetlenül hagyni, hogy több különböző adat kinyerése, hibakeresés, vagy ellenőrzés esetén vissza lehessen térni hozzá. Hagyományosan és esetünkben is a HTML kódot a <code>soup</code> változó tárolja.

Elsődleges célunk az egyes linkek kigyűjtése, amik <code>&lt;a href=&quot;&quot;&gt;</code> elemekben vannak tárolva.

In [6]:
articles = soup.find("div")
printLimited(articles,1000)

<div class="vector-header-container">
<header class="vector-header mw-header">
<div class="vector-header-start">
<nav aria-label="Wiki" class="vector-main-menu-landmark" role="navigation">
<div class="vector-dropdown vector-main-menu-dropdown vector-button-flush-left vector-button-flush-right" id="vector-main-menu-dropdown">
<input aria-haspopup="true" aria-label="Főmenü" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-vector-main-menu-dropdown" id="vector-main-menu-dropdown-checkbox" role="button" type="checkbox"/>
<label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-button--fake-button cdx-button--fake-button--enabled cdx-button--weight-quiet cdx-button--icon-only" for="vector-main-menu-dropdown-checkbox" id="vector-main-menu-dropdown-label"><span class="vector-icon mw-ui-icon-menu mw-ui-icon-wikimedia-menu"></span>
<span class="vector-dropdown-label-text">Főmenü</span>
</label>
<div class="vector-dropdown-content">
<div class="vector-unpinned-container

In [7]:
#mw-category mw-category-columns

articles = soup.find("div", class_="mw-category mw-category-columns")
printLimited(articles,500)

<div class="mw-category mw-category-columns"><div class="mw-category-group"><h3>A</h3>
<ul><li><a href="/wiki/Albatroszf%C3%A9l%C3%A9k" title="Albatroszfélék">Albatroszfélék</a></li>
<li><a href="/wiki/%C3%81lcsuszkaf%C3%A9l%C3%A9k" title="Álcsuszkafélék">Álcsuszkafélék</a></li>
<li><a href="/wiki/Alkaf%C3%A9l%C3%A9k" title="Alkafélék">Alkafélék</a></li>
<li><a href="/wiki/%C3%81lmosmad%C3%A1rf%C3%A9l%C3%A9k" title="Álmosmadárfélék">Álmosmadárfélék</a></li>
<li><a href="/wiki/Aptornithidae" titl


In [8]:
links = []
for link in articles.find_all('a'):
    links.append("https://hu.wikipedia.org"+link.get('href'))

print(links[0])

https://hu.wikipedia.org/wiki/Albatroszf%C3%A9l%C3%A9k


## Mappa létrehozása és törlése

Mappák létrehozása és ellenőrzése 2 alapkönyvtár bevonásával és feladattól függően több módon lehetséges.

Az <code>os</code> könyvtár képes fájlt és üres mappát törölni, valamint mappát létrehozni. Ehhez karakterlánc formájában a fájl/mappa nevét kell megadni, vagy az elérési útvonalát. Az <code>shutil</code> képes nem üres mappák törlésére is.

- <code>os.remove()</code> - megadott fájlt törli
- <code>os.rmdir()</code> - <u>üres</u> mappát töröl
- <code>shutil.rmtree()</code> - mappát töröl
- <code> os.makedirs()</code> - mappát hoz létre

Érdemes megjegyezni, hogy a törölni kívánt fájl/mappa nem létezik, akkor hibát fog jelezni. Ennek elkerülése érdekében érdemes először ellenőrizni, hogy létezik-e az adott fájl/mappa.
- <code> os.path.exists()</code> - ellenőrzi, hogy létezik-e az adott fájl/mappa


In [9]:
import os, shutil

def create_folder(folder_path):    
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        os.makedirs(folder_path)
        print(folder_path + " mappa kiürítve.")
    else:
        os.makedirs(folder_path)
        print(folder_path + " mappa létrehozva.")

folder_path = "images"
create_folder(folder_path)

images mappa létrehozva.


## Egy tétel címének, leírásának és képének letöltése

A szükséges részek elérésre CSS selector-okat használunk. Mivel a kép maga nem a HTML kód része, csak az elérési útvonala, ezért annak letöltése több lépésből áll:
- HTML struktúrából kinyerni a kép pontos elérési útvonalát
- kiegészíteni linkké
- <code>requests</code> segítségével lekérni a képet
- elmenteni a képet

In [10]:
url = "https://hu.wikipedia.org/wiki/Kazu%C3%A1rf%C3%A9l%C3%A9k"
response = re.get(url)
soup = BeautifulSoup(response.text, "html.parser")

title = soup.select("#firstHeading > span.mw-page-title-main")
description = soup.select("#mw-content-text > div.mw-content-ltr.mw-parser-output > p:nth-of-type(2)")
image = soup.select("#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody > tr:nth-child(3) > td > span > a > img ")
image_url = "https:"+image[0].get("src")

print(title[0].text)
print(description[0].text)
print(image[0].get("src"))

image_name = title[0].text + ".jpg"
page = re.get(image_url)

with open("images/"+image_name, 'wb') as f:
    f.write(page.content)

Kazuárfélék
Egyes rendszertanászok ide sorolják az emufélék (Dromaiidae) családját is Dromaiinae alcsaládként.

//upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Casuarius_casuariusAdjC1616.jpg/260px-Casuarius_casuariusAdjC1616.jpg


## Hibakezelés

Annak ellenére, hogy a célzott weboldalak hasonló struktúrát követnek, nem egyformák. Előfordulhat, hogy hiányzik a leírás, vagy épp a kép. Címmel az összes weboldal rendelkezik, viszont a többi adat esetén meg kell oldani a hibák kezelését. Ezt <code>try</code> és <code>except</code> blokkok beépítésével tudjuk elérni, a jelentkező hiba <code>IndexError</code> lehet.

In [11]:
data_file = open("data_birds.csv", "a", encoding="utf-8")

for item in range(0,len(links)):    
    response = re.get(links[item])
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.select("#firstHeading > span.mw-page-title-main")[0].text
    try:
        description = soup.find("div", {"class": "mw-content-ltr mw-parser-output"}).find_all("p")[2].text
    except IndexError:
        description = ""
    
    try:
        image = soup.select("#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody > tr:nth-child(3) > td > span > a > img ")
        image_url = "https:"+image[0].get("src")
        image_name = title + ".jpg"
        page = re.get(image_url)

        with open("images/"+image_name, 'wb') as f:
            f.write(page.content)
    except IndexError:
        pass
    
    data_file.write(str(title) + ',' + str(description))   
    print("|", end="")

data_file.close()      

|

|

|

|

|

...

|

|

|

|

KeyboardInterrupt: 

## Adatok kimentése változóba

Az előző kódot módosítva nem fájlba fogja kimenteni az adatokat, hanem változóba. Az, hogy fájlba, vagy változóba mentjük ki az adatokat attól függ, hogy továbbiakban milyen munka vár ránk.

In [29]:
data = []

for item in range(0, len(links)):
    response = re.get(links[item])
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.select("#firstHeading > span.mw-page-title-main")[0].text
    try:
        description = (
            soup.find("div", {"class": "mw-content-ltr mw-parser-output"})
            .find_all("p")[2]
            .text
        )
    except IndexError:
        description = ""

    try:
        image = soup.select(
            "#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody > tr:nth-child(3) > td > span > a > img "
        )
        image_url = "https:" + image[0].get("src")
        image_name = title + ".jpg"
        page = re.get(image_url)

        with open("images/" + image_name, "wb") as f:
            f.write(page.content)
    except IndexError:
        image_url = ""

    data.append([title, description, image_url, links[item]])
    print("|", end="") 

print("\n" + str(data[5]))    

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
['Ásótyúkfélék', 'Az ásótyúkfélék (Megapodiidae) a madarak osztályába és a tyúkalakúak (Galliformes) rendjébe tartozó család. Egyéb elnevezései: talegallatyúk-félék, halomköltő tyúkok, lábastyúkok.\n', 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Alectura_lathami.jpg/260px-Alectura_lathami.jpg', 'https://hu.wikipedia.org/wiki/%C3%81s%C3%B3ty%C3%BAkf%C3%A9l%C3%A9k']


## HTML weboldal készítése a begyűjtött adatokból

Egy sablon kialakításával képesek vagyunk a gyűjtött adatokat felhasználni egy egyszerű weboldalt létrehozni. Előfordulhat, hogy többszöri lekérdezések miatt a képeket letöltését megakadályozza a webhely, ezt a <code>requests</code> lekérés <code>header=""</code> paraméterével tudjuk elkerülni.

In [30]:
headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
}

with open("index_birds.html", "w", encoding = "utf-8") as f:
    f.write("<!DOCTYPE html>\n")
    f.write("<html>\n")
    f.write("<head>\n")
    f.write("<title>Madárcsaládok</title>\n")
    f.write("</head>\n")
    f.write("<body>\n")

    for title, description, img_url, url in data:
        f.write(f'<h1>{title}</h1>\n')        
        f.write(f'<p>{description}</p>\n')
        f.write(f'<a href={url}> forrás </a><br>')

        try:
            page = re.get(img_url, headers=headers) 
            if page.status_code == 200:                
                with open("images/"+title+".jpg", 'wb') as g:
                    g.write(page.content)                    
            else:
                print(page.status_code)
            
            image_path = "images/" + title + ".jpg"
                        
            f.write(f'<img src="{image_path}" alt="{title}"><br>')
            f.write("<hr>")            
        except re.exceptions.MissingSchema:
            pass          
            
    f.write("</body>\n")
    f.write("</html>\n")